In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import BertModel, BertConfig
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

c:\Users\Manann\anaconda3\envs\fitness-ml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Load the CSV file
df = pd.read_csv("fitness_keypoints_full.csv")

# Drop any rows with NaN values (if present)
df.dropna(inplace=True)

# Extract features (pose keypoints) and labels
X = df.iloc[:, :-1].values  # All columns except the last one (features)
y = df.iloc[:, -1].values   # Last column (exercise label)

# Encode exercise labels into numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split into training & testing (70-10-20 split)
X_train, X_temp, y_train, y_temp = train_test_split(X, y_encoded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.67, random_state=42)

# Convert to PyTorch tensors
X_train, X_val, X_test = map(torch.tensor, (X_train, X_val, X_test))
y_train, y_val, y_test = map(torch.tensor, (y_train, y_val, y_test))

print(f"Training set: {X_train.shape}, Validation set: {X_val.shape}, Test set: {X_test.shape}")

Training set: torch.Size([11132, 100]), Validation set: torch.Size([1574, 100]), Test set: torch.Size([3198, 100])


In [8]:
class PoseBERT(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(PoseBERT, self).__init__()
        config = BertConfig(
            hidden_size=input_dim,  # Match to input feature size (100)
            num_hidden_layers=4,
            num_attention_heads=4,
            intermediate_size=512
        )
        self.bert = BertModel(config)
        self.fc = nn.Linear(input_dim, num_classes)  # Change from 256 → input_dim

    def forward(self, x):
        outputs = self.bert(inputs_embeds=x)
        x = outputs.last_hidden_state[:, 0, :]  # Extract first token embedding
        x = self.fc(x)  # Ensure input size matches FC layer
        return x


# Initialize model
num_classes = len(label_encoder.classes_)
model = PoseBERT(input_dim=X_train.shape[1], num_classes=num_classes).to("cuda" if torch.cuda.is_available() else "cpu")

# Define loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
# Training Loop
y_train, y_val, y_test = map(lambda y: y.long(), (y_train, y_val, y_test))
num_epochs = 100
device = "cuda" if torch.cuda.is_available() else "cpu"

model.train()
for epoch in range(num_epochs):
    total_loss = 0.0
    for i in tqdm(range(len(X_train))):
        inputs = X_train[i].float().unsqueeze(0).unsqueeze(0).to(device)  
        labels = y_train[i].unsqueeze(0).to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(X_train):.4f}")

print("Training Complete!")


100%|██████████| 11132/11132 [05:49<00:00, 31.85it/s]


Epoch 1/100, Loss: 2.6811


100%|██████████| 11132/11132 [05:26<00:00, 34.09it/s]


Epoch 2/100, Loss: 2.6567


100%|██████████| 11132/11132 [05:24<00:00, 34.29it/s]


...

Epoch 65/100, Loss: 2.6489


100%|██████████| 11132/11132 [06:50<00:00, 27.12it/s] 


Epoch 66/100, Loss: 2.6488


 72%|███████▏  | 7997/11132 [04:26<01:44, 30.03it/s]


KeyboardInterrupt: 

In [10]:
# Evaluation
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for i in tqdm(range(len(X_test))):
        inputs = X_test[i].float().unsqueeze(0).to(device)
        labels = y_test[i].unsqueeze(0).to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        y_pred.append(predicted.cpu().numpy()[0])
        y_true.append(labels.cpu().numpy()[0])

# Compute F1-score & Classification Report
from sklearn.metrics import classification_report

print("Classification Report:\n", classification_report(y_true, y_pred, target_names=label_encoder.classes_))

# Compute & Plot Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("PoseBERT Confusion Matrix")
plt.show()

  0%|          | 0/3198 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Load preprocessed pose keypoints data
df = pd.read_csv("fitness_keypoints_full.csv")

# Extract keypoints (features) and labels
X = df.iloc[:, :-1].values  # Pose keypoints
y = df.iloc[:, -1].values   # Exercise labels

# Encode labels into integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float)
y_encoded = torch.tensor(y_encoded, dtype=torch.long)

# Define body edges (connections between joints)
edges = torch.tensor([
    [0, 1], [1, 2], [2, 3],  # Left arm
    [0, 4], [4, 5], [5, 6],  # Right arm
    [0, 7], [7, 8], [8, 9],  # Spine
    [8, 10], [10, 11], [11, 12],  # Left leg
    [8, 13], [13, 14], [14, 15]   # Right leg
], dtype=torch.long).T  # PyTorch Geometric expects (2, num_edges)


In [ ]:
# Create graph dataset
graph_data = []
for i in range(len(X)):
    data = Data(x=X[i].view(-1, 1), edge_index=edges, y=y_encoded[i])
    graph_data.append(data)

# Split into train and test sets
train_size = int(0.7 * len(graph_data))
val_size = int(0.1 * len(graph_data))
test_size = len(graph_data) - train_size - val_size

train_data, val_data, test_data = torch.utils.data.random_split(graph_data, [train_size, val_size, test_size])

# Create DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)

In [ ]:
class PoseGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(PoseGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, data.batch)  # Aggregate node features
        x = self.fc(x)
        return x

# Initialize model
num_classes = len(label_encoder.classes_)
model = PoseGNN(input_dim=1, hidden_dim=64, num_classes=num_classes).to("cpu")

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training Loop
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for data in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

print("Training Complete!")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Evaluate Model
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for data in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(data.y.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Compute Classification Report
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=label_encoder.classes_))

# Compute Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("GNN Confusion Matrix")
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
import cv2
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Block 1: Load ResNet Model for Feature Extraction
resnet = models.resnet18(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove last FC layer
resnet.eval().to(device)

# Define transformation for input frames
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [ ]:
def extract_video_frames(video_path, sequence_length=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert to RGB and preprocess
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = transform(frame)
        frames.append(frame)
        
        # Limit number of frames
        if len(frames) >= sequence_length:
            break
    
    cap.release()
    
    # Pad sequence if too short
    while len(frames) < sequence_length:
        frames.append(frames[-1])
    
    return torch.stack(frames).to(device)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim=512, hidden_dim=128, num_classes=10):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])
        return out

# Initialize model
num_classes = 10  # Adjust based on dataset
model = LSTMModel(input_dim=512, hidden_dim=128, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
lass VideoDataset(Dataset):
    def __init__(self, video_dir, labels, sequence_length=30):
        self.video_dir = video_dir
        self.labels = labels
        self.sequence_length = sequence_length
        self.video_files = list(labels.keys())
    
    def __len__(self):
        return len(self.video_files)
    
    def __getitem__(self, idx):
        video_path = os.path.join(self.video_dir, self.video_files[idx])
        frames = extract_video_frames(video_path, self.sequence_length)
        label = torch.tensor(self.labels[self.video_files[idx]], dtype=torch.long)
        
        # Extract ResNet features
        with torch.no_grad():
            features = resnet(frames).squeeze(-1).squeeze(-1)  # Shape: (sequence_length, 512)
        
        return features, label

In [ ]:
video_dir = "path_to_videos"  # Update this
labels = {"video1.mp4": 0, "video2.mp4": 1}  # Example label dictionary
train_dataset = VideoDataset(video_dir, labels)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

print("Training Complete!")